# Spring 2023 : CS5720
### Neural Networks & Deep Learning ICP_10 : Jahnavi Chadalavada (700728443)

In [98]:
from google.colab import drive 
drive.mount('/NN') 

Drive already mounted at /NN; to attempt to forcibly remount, call drive.mount("/NN", force_remount=True).


In [99]:
path_to_csv = '/NN/MyDrive/Colab Notebooks/NN/Sentiment.csv'

In [100]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [101]:
data = pd.read_csv(path_to_csv)

In [61]:
data.head(5)

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [102]:
# Keeping only the neccessary columns
data = data[['text','sentiment']]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

<ipython-input-102-c56dd62cb117>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())


In [103]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

In [104]:
X = tf.keras.utils.pad_sequences(X)
X.shape

(13871, 28)

In [105]:
embed_dim = 128
lstm_out = 196

In [106]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model

In [107]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()

In [108]:
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)

291/291 - 35s - loss: 0.8259 - accuracy: 0.6445 - 35s/epoch - 120ms/step


In [109]:
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)

144/144 - 2s - loss: 0.7728 - accuracy: 0.6704 - 2s/epoch - 17ms/step


In [110]:
print(score)
print(acc)
print(model.metrics_names)

0.7728039026260376
0.6703800559043884
['loss', 'accuracy']


In [23]:
model.save("/NN/MyDrive/Colab Notebooks/NN/LSTM_Twitter.h5")

In [46]:
model = tf.keras.models.load_model('/NN/MyDrive/Colab Notebooks/NN/LSTM_Twitter.h5')

## Prediction on new text data

In [96]:
sentence = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"]
s = list(map(lambda x: x.lower(),sentence))
s = list(map((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)),s))
s = list(map(lambda x: x.replace('rt', ' '),s))

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(s)
input = tokenizer.texts_to_sequences(s)
input = tf.keras.utils.pad_sequences(input)

In [81]:
prediction = model.predict(input)

1/1 [==============================] - 0s 27ms/step


In [82]:
print(prediction)

[[0.7791218  0.13369924 0.08717896]]


In [83]:
import numpy as np
pred_labels = []
for i in prediction:
    print(i,np.max(i))
    if np.max(i) >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
for i in range(len(sentence)):
    print(sentence[i])
    if pred_labels[i] == 1:
        s = 'Positive'
    else:
        s = 'Negative'
    print("Predicted sentiment : ",s)

[0.7791218  0.13369924 0.08717896] 0.7791218
A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump
Predicted sentiment :  Positive


##GridSearchCV

In [93]:
from scikeras.wrappers import KerasClassifier
model = KerasClassifier(build_fn=model,verbose=0)
batch_size = [10,20, 40]
epochs = [1, 2, 3]
param_grid = dict(batch_size=batch_size, epochs=epochs)


In [94]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=2)
grid_result = grid.fit(X_train, Y_train)

/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/scikeras/wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``

In [95]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.651111 using {'batch_size': 20, 'epochs': 2}
